In [2]:
import vectorbt as vbt
import numpy as np
import pandas as pd
import yfinance as yf
from numba import njit
import ta

In [24]:

# Data
symbol = 'IWM'  # Using QQQ as an example

price = vbt.YFData.download(
    symbol,
    start='1/1/2018',
    interval='1d'
).get('Close')

fast_level = np.arange(9,27)
slow_level = np.arange(26,60)
EMA = vbt.IndicatorFactory.from_ta('EMAIndicator')
ema13 = EMA.run(price, fast_level)
ema48 = EMA.run(price, slow_level)

windows = np.arange(2, 101)
ema13, ema48 = EMA.run_combs(price, window=windows, r=2, short_names=['fast', 'slow'])

entries = ema13.ema_indicator_crossed_above(ema48)
exits = ema48.ema_indicator_crossed_below(ema48)

@njit
def adjust_sl_func_nb(c):
    current_profit = (c.val_price_now - c.init_price) / c.init_price
    if current_profit >= 0.6:
        return 0.4, True
    if current_profit >= 0.40:
        return 0.05, True
    elif current_profit >= 0.2:
        return 0.01, True
    # elif current_profit >= 0.05:
    #     return 0.01, True
    # else:
    #     return 0.2, False
    return c.curr_stop, c.curr_trail

take_profit_pct = 20  # 10%
stop_loss_pct = 0.1  # -5%

# Backtest with vectorbt, specifying short entries and exits
pf = vbt.Portfolio.from_signals(
    close=price,
    entries=entries,
    exits=exits,
    short_entries=exits,
    short_exits=entries,
    init_cash=10000,
    size=1,
    size_type=vbt.portfolio.enums.SizeType.Percent,
    sl_stop=stop_loss_pct,
    adjust_sl_func_nb=adjust_sl_func_nb,
    tp_stop=take_profit_pct,
    freq='1D'
)

In [25]:
pf_kwargs = dict(size=np.inf, fees=0.001, freq='1D')

fig = pf.total_return().vbt.heatmap(
    x_level='fast_window', y_level='slow_window', symmetric=True,
    trace_kwargs=dict(colorbar=dict(title='Total return', tickformat='%')))
fig.show()

d:\Projects\breeze_bt\.venv\Lib\site-packages\jupyter_client\session.py:721: UserWarning:

Message serialization failed with:
Out of range float values are not JSON compliant: nan
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant



In [27]:
# Plot cumulative returns
# pf.cumulative_returns().vbt.plot().show()
opf = pf[(2,3)]
opf.plot().show()


# Show performance metrics
print(opf.stats())


Start                         2018-01-02 05:00:00+00:00
End                           2024-03-28 04:00:00+00:00
Period                               1570 days 00:00:00
Start Value                                     10000.0
End Value                                  16994.160313
Total Return [%]                              69.941603
Benchmark Return [%]                          48.108366
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              33.286613
Max Drawdown Duration                 599 days 00:00:00
Total Trades                                         11
Total Closed Trades                                  10
Total Open Trades                                     1
Open Trade PnL                                53.334322
Win Rate [%]                                       60.0
Best Trade [%]                                23.863929
Worst Trade [%]                              -17